The oracle of the exact distance

In [6]:
import json
import pennylane as qml
import pennylane.numpy as np

In [7]:
wires_m = [0, 1, 2]  # qubits needed to encode m
wires_n = [3, 4, 5]  # qubits needed to encode n
wires_aux = [6, 7, 8, 9, 10]  # auxiliary qubits you can use

In [8]:
def controlled_Z_gate(wires):
    qml.CZ(wires=wires)


In [9]:
def oracle_distance(d):
    """
    Args:
        d (int): the distance with which we will check that the oracle is working properly.

    This function does not return anything, it is a quantum function that applies
    necessary gates that implement the requested oracle.

    """


    # Encode m and n into auxiliary qubits
    qml.CNOT(wires=[0, 6])
    qml.CNOT(wires=[1, 7])
    qml.CNOT(wires=[2, 8])
    qml.CNOT(wires=[3, 9])
    qml.CNOT(wires=[4, 10])
    
    # Check the distance between m and n
    qml.CNOT(wires=[6, 7])
    qml.CNOT(wires=[7, 8])
    qml.CNOT(wires=[8, 9])
    qml.CNOT(wires=[9, 10])
    qml.CNOT(wires=[10, 6])
    
    # Apply phase flip if distance equals d
    qml.Toffoli(wires=[6, 7, 11])
    qml.Toffoli(wires=[7, 8, 11])
    qml.Toffoli(wires=[8, 9, 11])
    qml.Toffoli(wires=[9, 10, 11])
    qml.Toffoli(wires=[10, 6, 11])
    
    qml.PauliX(wires=11).controlled(wires=[6, 7, 8, 9, 10])
    
    # Reverse the encoding of m and n
    qml.CNOT(wires=[4, 10])
    qml.CNOT(wires=[3, 9])
    qml.CNOT(wires=[2, 8])
    qml.CNOT(wires=[1, 7])
    qml.CNOT(wires=[0, 6])

In [10]:
# These functions are responsible for testing the solution.
wires_m = [0, 1, 2]
wires_n = [3, 4, 5]
wires_aux = [6, 7, 8, 9, 10]

dev = qml.device("default.qubit.legacy", wires=11)


@qml.qnode(dev)
def circuit(m, n, d):
    qml.BasisEmbedding(m, wires=wires_m)
    qml.BasisEmbedding(n, wires=wires_n)
    oracle_distance(d)
    return qml.state()


def run(test_case_input: str) -> str:
    outputs = []
    d = int(json.loads(test_case_input))
    for n in range(8):
        for m in range(8):
            outputs.append(sum(circuit(n, m, d)).real)
    output_list = [round(elem.numpy()) for elem in outputs]
    return str(output_list)


def check(solution_output: str, expected_output: str) -> None:
    i = 0
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output)

    circuit(np.random.randint(7), np.random.randint(7), np.random.randint(7))
    tape = circuit.qtape

    names = [op.name for op in tape.operations]

    assert names.count("QubitUnitary") == 0, "Can't use custom-built gates!"

# These are the public test cases
test_cases = [
    ('0', '[-1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1]'),
    ('1', '[1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1]'),
    ('2', '[1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1]'),
    ('3', '[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1]'),
    ('4', '[1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1]'),
    ('5', '[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1]'),
    ('6', '[1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1]'),
    ('7', '[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '0'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 1 with input '1'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 2 with input '2'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 3 with input '3'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 4 with input '4'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 5 with input '5'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 6 with input '6'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
Running test case 7 with input '7'...
Runtime Error. 'PauliX' object has no attribute 'controlled'
